# Tensorflow MNIST (using CNN)

## <font color='dark'>Import</font>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.examples.tutorials.mnist.input_data as input_data
%matplotlib inline

## <font color='dark'>Load data</font>

In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## <font color='dark'>Build the Function used in model</font>

In [3]:
# 隨機方式初始化我們的 權重(weight)，使用 tf.truncated_normal, 指定標準差為 0.1

def weight(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1), name='W')

In [4]:
# 使用 tf.constant 建立常數，再用 tf.variable 建立變數回傳

def bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape), name='b')

In [5]:
# 進行卷積運算，相當於濾鏡的意思
# x 是輸入的影像; W 是 filter 的權重;
# strides 為濾鏡每次移動的格數，格式為 [1,由左而右移動，由上而下移動,1]; padding 就是當 filter 超過邊界，則空缺處補 0 運算

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [6]:
# 建立 max pooling 作池化層
# ksize 是窗口的大小，像現在的是 2*2，格式為 [1,height, weight, 1]
# stride 為每次窗口移動的距離，像現在是 左而右、上而下 移動 2 格，格式為 [1,stride, stride, 1]

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

## <font color='dark'>Build the model</font>

**建立輸入層**

In [7]:
with tf.name_scope('Input_layer'):
    x = tf.placeholder('float',shape=[None,784],name='x')
    x_image = tf.reshape(x, [-1,28,28,1])   # 原本是 1 維向量，因為要進行 卷積 與 池化 運算，必須轉成 4 維向量

**建立卷積層 1**

In [8]:
with tf.name_scope('C1_Conv'):
    W1 = weight([5,5,1,16])          # 建立 filter weight，大小是 5*5，因為是單色只有一張因此為 1，16指要產生 16張影像
    b1 = bias([16])                  # 建立 b1 偏差值，因為 Conv1 要產生 16 張影像，所以他的 shape 為 [16]
    Conv1 = conv2d(x_image, W1)+b1   # 針對影像進行 卷積運算
    C1_conv = tf.nn.relu(Conv1)      # 最後使用 relu 函數轉換我們的結果

**建立池化層 1**

In [9]:
with tf.name_scope('C1_Pool'):
    C1_Pool = max_pool_2x2(C1_conv)

**建立卷積層 2**

In [10]:
with tf.name_scope('C2_Conv'):
    W2 = weight([5,5,16,36])          # 建立 filter weight，大小是 5*5，單色為 1，16指要產生 16張影像
    b1 = bias([36])                  # 建立 b1 偏差值，因為 Conv1 要產生 16 張影像，所以他的 shape 為 [16]
    Conv2 = conv2d(C1_Pool, W2)+b1   # 針對影像進行 卷積運算
    C2_conv = tf.nn.relu(Conv2)      # 最後使用 relu 函數轉換我們的結果

**建立池化層 2**

In [11]:
with tf.name_scope('C2_Pool'):
    C2_Pool = max_pool_2x2(C2_conv)

**建立平坦層**

In [12]:
with tf.name_scope('D_Flat'):
    D_Flat = tf.reshape(C2_Pool,[-1,1764])    # 1764 = 36*7*7，第一個參數為 -1 : 因為之後會放入不限定筆數的資料

**建立隱藏層**

In [13]:
with tf.name_scope('D_Hidden_Layer'):
    W3 = weight([1764,128])                                     # 上層是 1764 進來，然後 hidden layer 的神經元我們設 128
    b3 = bias([128])
    D_Hidden = tf.nn.relu(tf.matmul(D_Flat,W3)+b3)              # D_Flat 和 W3 矩陣相乘，再加上 偏差值
    D_Hidden_Dropout = tf.nn.dropout(D_Hidden,keep_prob=0.8)    # keep_prob 為隨機保留的比率，全保留為 1

**建立輸出層**

In [14]:
with tf.name_scope('Output_Layer'):
    W4 =weight([128,10])
    b4 = bias([10])
    y_predict = tf.nn.softmax(tf.matmul(D_Hidden_Dropout,W4)+b4)

## <font color='dark'>Define the training way</font>

In [15]:
with tf.name_scope("optimizer"):
    y_label = tf.placeholder("float",shape=[None,10],name="y_label")
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predict,labels=y_label))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)

## <font color='dark'>Define the evaluation way</font>

In [16]:
with tf.name_scope("evaluate_model"):
    correct_prediction = tf.equal(tf.argmax(y_predict,1),tf.argmax(y_label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

## <font color='dark'>Training the model</font>

In [17]:
trainEpochs = 30
batchSize = 100
totalBatchs = int(mnist.train.num_examples/batchSize)

epoch_list=[];accuracy_list=[];loss_list=[];

from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
for epoch in range(trainEpochs):
    for i in range(totalBatchs):
        batch_x, batch_y = mnist.train.next_batch(batchSize)
        sess.run(optimizer,feed_dict={x:batch_x,y_label:batch_y})
    
    loss,acc = sess.run([loss_function,accuracy],feed_dict={x:mnist.validation.images,y_label:mnist.validation.labels})
    
    epoch_list.append(epoch)
    loss_list.append(loss)
    accuracy_list.append(acc)
    
    print('Train Epoch : ',\
          '%0.2d'%(epoch+1),\
          "Loss : ",\
          "%0.5f"%(loss),\
          "Accuracy : ",acc)

duration = time()-startTime
print("Train Finished takes : %0.3f sec"%duration)

Train Epoch :  01 Loss :  1.58684 Accuracy :  0.8992
Train Epoch :  02 Loss :  1.54115 Accuracy :  0.9322
Train Epoch :  03 Loss :  1.51773 Accuracy :  0.9524
Train Epoch :  04 Loss :  1.50936 Accuracy :  0.9588
Train Epoch :  05 Loss :  1.50117 Accuracy :  0.9646
Train Epoch :  06 Loss :  1.49771 Accuracy :  0.96860003
Train Epoch :  07 Loss :  1.49441 Accuracy :  0.9710001
Train Epoch :  08 Loss :  1.49152 Accuracy :  0.97340006
Train Epoch :  09 Loss :  1.48895 Accuracy :  0.9746
Train Epoch :  10 Loss :  1.48934 Accuracy :  0.97400004
Train Epoch :  11 Loss :  1.48724 Accuracy :  0.9768
Train Epoch :  12 Loss :  1.48459 Accuracy :  0.97840005
Train Epoch :  13 Loss :  1.48488 Accuracy :  0.97800004
Train Epoch :  14 Loss :  1.48402 Accuracy :  0.9796001
Train Epoch :  15 Loss :  1.48155 Accuracy :  0.98060006
Train Epoch :  16 Loss :  1.48048 Accuracy :  0.98200005
Train Epoch :  17 Loss :  1.48151 Accuracy :  0.98100007
Train Epoch :  18 Loss :  1.47938 Accuracy :  0.98440003
Trai

## <font color='dark'>Show the accuracy</font>

In [20]:
print("Accuracy : ",sess.run(accuracy,feed_dict={x:mnist.test.images,y_label:mnist.test.labels}))

Accuracy :  0.98730004


## <font color='dark'>Predict the result</font>

In [22]:
prediction_result = sess.run( tf.argmax(y_predict,1),feed_dict={x:mnist.test.images}  )

In [23]:
prediction_result[:10]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int64)

## <font color='dark'>Tensorboard</font>

In [24]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('log/CNN',sess.graph)